In [27]:
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import yfinance as yf
from panel.interact import interact
import plotly.express as px
import panel as pn
pn.extension('plotly')
import seaborn as sns

In [78]:
ticker = "FBHS"

In [79]:
df_data = yf.download(ticker,
start="2020-10-01",
end="2021-04-14",
progress=False,
interval = "1d",
index_col = "Date",
infer_datetime_format= True,
parse_dates = True,                    
groupby = "column"
                )

In [80]:
signals_df = df_data.drop(columns = ["Open","Adj Close","Volume"])

In [81]:
from talib import ATR
def ATR_Trailing_Stop (high, low, close, period, factor):
    
    atr_val = ATR(high, low, close, timeperiod=period)
    trail = atr_val * factor
    
    prev = close[period] - trail[period]
    
    for i in range(period+1,trail.size):
        if (close[i] > prev) and (close[i-1] > prev):
            trail[i] = max(prev, close[i] - trail[i])
        elif (close[i] < prev) and (close[i-1] < prev):
            trail[i] = min(prev, close[i] + trail[i])
        elif close[i] > prev:
            trail[i] = close[i] - trail[i]
        else:
            trail[i] = close[i] + trail[i]
        prev = trail[i]
    return trail


In [82]:
def calc_signal (buy, sell):
    position = 0
    signals = pd.Series(index = buy.index, dtype = float)
    
    for i in range(0, signals.size):
        if position == 0 and buy[i] == 1:
            signals[i] = 1
            position = 1
        elif position == 1 and sell[i] == 1:
            signals[i] = -1
            position = 0
        else:
            signals[i] = 0
        
    return signals

In [83]:
from talib import ATR

#Define Window 13,34,89 Fibonanci numbers
EMAshort = 13
EMAmid = 34
EMAlong = 89

#Generate EMA
signals_df["EMAshort"] = signals_df["Close"].ewm(halflife=EMAshort).mean()
signals_df["EMAmid"] = signals_df["Close"].ewm(halflife=EMAmid).mean()
signals_df["EMAlong"] = signals_df["Close"].ewm(halflife=EMAlong).mean()
signals_df["Entry"] = 0.0
signals_df["Exit"] = 0.0

#Generate ATR Trailing Stop
close = signals_df.Close
high = signals_df.High
low = signals_df.Low
signals_df["ATRts"] = ATR_Trailing_Stop (high, low, close, 89, 10)

# Generate the trading signal
signals_df["Entry"][EMAshort:] = np.where(
    signals_df["EMAmid"][EMAshort:] - signals_df["EMAlong"][EMAshort:] > 0, 1.0, 0.0
)
signals_df["Exit"][EMAshort:] = np.where(
    signals_df["ATRts"][EMAshort:] - signals_df["Close"][EMAshort:] > 0, 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Buy"] = signals_df["Entry"].diff()
signals_df["Sell"] = signals_df["Exit"].diff()
signals_df["Entry/Exit"] = calc_signal (signals_df["Buy"], signals_df["Sell"])
signals_df.head()

,High,Low,Close,EMAshort,EMAmid,EMAlong,Entry,Exit,ATRts,Buy,Sell,Entry/Exit
Date,,,,,,,,,,,,
2020-10-01,88.209999,86.129997,87.669998,87.669998,87.669998,87.669998,0.0,0.0,NaN,NaN,NaN,0.0
2020-10-02,88.070000,85.269997,87.519997,87.592998,87.594233,87.594705,0.0,0.0,NaN,0.0,0.0,0.0
2020-10-05,88.879997,88.070000,88.809998,88.020476,88.007777,88.002962,0.0,0.0,NaN,0.0,0.0,0.0
2020-10-06,89.559998,87.070000,87.180000,87.793266,87.794462,87.794812,0.0,0.0,NaN,0.0,0.0,0.0
2020-10-07,88.599998,86.800003,87.599998,87.750385,87.753968,87.755240,0.0,0.0,NaN,0.0,0.0,0.0


In [84]:
def calc_holdings (EntryExit):
    holding = 0
    signals = pd.Series(index = EntryExit.index, dtype = float)
    
    for i in range(0, EntryExit.size):
        holding = holding + EntryExit[i]
        signals[i] = holding
        
    return signals

In [85]:
signals_df['holding'] = calc_holdings(signals_df["Entry/Exit"])

In [86]:
# Set initial capital
initial_capital = signals_df[signals_df['Entry/Exit']==1].iloc[0]['Close']*1000

# Set the share size
share_size = 1000

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['holding']

# Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail()

,High,Low,Close,EMAshort,EMAmid,EMAlong,Entry,Exit,ATRts,Buy,Sell,Entry/Exit,holding,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,,,,
2021-04-07,98.809998,97.739998,98.360001,91.212725,88.936050,87.869248,1.0,0.0,72.676307,0.0,0.0,0.0,1.0,1000.0,0.0,98360.000610,0.0,98360.000610,-0.001928,0.122447
2021-04-08,98.800003,97.769997,98.440002,91.588350,89.142419,87.998053,1.0,0.0,72.909866,0.0,0.0,0.0,1.0,1000.0,0.0,98440.002441,0.0,98440.002441,0.000813,0.123360
2021-04-09,101.599998,98.290001,100.690002,92.061368,89.392779,88.152023,1.0,0.0,75.074812,0.0,0.0,0.0,1.0,1000.0,0.0,100690.002441,0.0,100690.002441,0.022857,0.149036
2021-04-12,101.760002,100.669998,101.019997,92.526931,89.644488,88.307449,1.0,0.0,75.570145,0.0,0.0,0.0,1.0,1000.0,0.0,101019.996643,0.0,101019.996643,0.003277,0.152802
2021-04-13,100.849998,98.709999,99.910004,92.910598,89.866393,88.446987,1.0,0.0,75.570145,0.0,0.0,0.0,1.0,1000.0,0.0,99910.003662,0.0,99910.003662,-0.010988,0.140135


In [87]:
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)



In [88]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': ticker,
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [89]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize Indicators
indicators = signals_df[['EMAmid', 'EMAlong','ATRts']].hvplot(
    ylabel='Price in $',
    width=750,
    height=300
)


# Overlay plots
entry_exit_plot = security_close * indicators * entry * exit
entry_exit_plot.opts(title=f'{ticker}: EMA & ATR Trailing Stop Entry/Exit', xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [90]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(title=f"{ticker}: EMA & ATR Trailing Stop Portfolio Value", xaxis=None)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [91]:
# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["SMASignal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["SMASignal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["SMAEntry/Exit"] = signals_df["SMASignal"].diff()

In [92]:
# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['SMA Position'] = share_size * signals_df['SMASignal']

# Find the points in time where a 500 share position is bought or sold
signals_df['SMA Entry/Exit Position'] = signals_df['SMA Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['SMA Portfolio Holdings'] = signals_df['Close'] * signals_df['SMA Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['SMA Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['SMA Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['SMA Portfolio Total'] = signals_df['SMA Portfolio Cash'] + signals_df['SMA Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['SMA Portfolio Daily Returns'] = signals_df['SMA Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['SMA Portfolio Cumulative Returns'] = (1 + signals_df['SMA Portfolio Daily Returns']).cumprod() - 1


In [93]:
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Algo1 Backtest','SMA Backtest']

portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

In [94]:
portfolio_evaluation_df.loc['Cumulative Returns']['Algo1 Backtest'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df.loc['Annual Return']['Algo1 Backtest'] = (signals_df['Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Volatility']['Algo1 Backtest'] = (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Sharpe Ratio']['Algo1 Backtest'] = (signals_df['Portfolio Daily Returns'].mean() * 252) / (signals_df['Portfolio Daily Returns'].std() * np.sqrt(252))

portfolio_evaluation_df.loc['Cumulative Returns']['SMA Backtest'] = signals_df['SMA Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df.loc['Annual Return']['SMA Backtest'] = (signals_df['SMA Portfolio Daily Returns'].mean() * 252)
portfolio_evaluation_df.loc['Annual Volatility']['SMA Backtest'] = (signals_df['SMA Portfolio Daily Returns'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Sharpe Ratio']['SMA Backtest'] = (signals_df['SMA Portfolio Daily Returns'].mean() * 252) / (signals_df['SMA Portfolio Daily Returns'].std() * np.sqrt(252))

#Algo1 Sortino Ratio
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

#SMA Sortino Ratio
SMA_sortino_ratio_df = signals_df[['SMA Portfolio Daily Returns']].copy()
SMA_sortino_ratio_df.loc[:,'SMA Downside Returns'] = 0

#Algo1 Downside Returns
target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

#SMA Downside Returns
SMA_mask = SMA_sortino_ratio_df['SMA Portfolio Daily Returns'] < target
SMA_sortino_ratio_df.loc[mask, 'SMA Downside Returns'] = SMA_sortino_ratio_df['SMA Portfolio Daily Returns']**2

#Also1 Sortino Raio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev
portfolio_evaluation_df.loc['Sortino Ratio']['Algo1 Backtest'] = sortino_ratio

#SMA Sortino Raio
SMA_down_stdev = np.sqrt(SMA_sortino_ratio_df['SMA Downside Returns'].mean()) * np.sqrt(252)
SMA_expected_return = SMA_sortino_ratio_df['SMA Portfolio Daily Returns'].mean() * 252
SMA_sortino_ratio = SMA_expected_return/SMA_down_stdev
portfolio_evaluation_df.loc['Sortino Ratio']['SMA Backtest'] = SMA_sortino_ratio

portfolio_evaluation_df

,Algo1 Backtest,SMA Backtest
Annual Return,0.301565,0.271115
Cumulative Returns,0.140135,0.144471
Annual Volatility,0.313307,0.151862
Sharpe Ratio,0.962522,1.78527
Sortino Ratio,1.36982,3.04457


In [95]:
# Visualize exit position relative to close price
SMAexit = signals_df[signals_df['SMAEntry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize entry position relative to close price
SMAentry = signals_df[signals_df['SMAEntry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize close price for the investment
SMAsecurity_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=750,
    height=300
)

# Visualize moving averages
SMAmoving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=750,
    height=300
)

# Overlay plots
SMAentry_exit_plot = SMAsecurity_close * SMAmoving_avgs * SMAentry * SMAexit
SMAentry_exit_plot.opts(title=f'{ticker}: SMA Entry/Exit', xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [96]:
# Visualize exit position relative to total portfolio value
SMAexit = signals_df[signals_df['SMAEntry/Exit'] == -1.0]['SMA Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Visualize entry position relative to total portfolio value
SMAentry = signals_df[signals_df['SMAEntry/Exit'] == 1.0]['SMA Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Visualize total portoflio value for the investment
SMAtotal_portfolio_value = signals_df[['SMA Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=750,
    height=300
)

# Overlay plots
SMAportfolio_entry_exit_plot = SMAtotal_portfolio_value * SMAentry * SMAexit
SMAportfolio_entry_exit_plot.opts(title=f'{ticker}: SMA Portfolio Value',xaxis=None)

:Overlay
   .Curve.I                        :Curve   [Date]   (SMA Portfolio Total)
   .Scatter.SMA_Portfolio_Total.I  :Scatter   [Date]   (SMA Portfolio Total)
   .Scatter.SMA_Portfolio_Total.II :Scatter   [Date]   (SMA Portfolio Total)

In [97]:
signals_df.head()

,High,Low,Close,EMAshort,EMAmid,EMAlong,Entry,Exit,ATRts,Buy,...,SMA100,SMASignal,SMAEntry/Exit,SMA Position,SMA Entry/Exit Position,SMA Portfolio Holdings,SMA Portfolio Cash,SMA Portfolio Total,SMA Portfolio Daily Returns,SMA Portfolio Cumulative Returns
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-01,88.209999,86.129997,87.669998,87.669998,87.669998,87.669998,0.0,0.0,NaN,NaN,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-02,88.070000,85.269997,87.519997,87.592998,87.594233,87.594705,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,87629.997253,87629.997253,NaN,NaN
2020-10-05,88.879997,88.070000,88.809998,88.020476,88.007777,88.002962,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,87629.997253,87629.997253,0.0,0.0
2020-10-06,89.559998,87.070000,87.180000,87.793266,87.794462,87.794812,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,87629.997253,87629.997253,0.0,0.0
2020-10-07,88.599998,86.800003,87.599998,87.750385,87.753968,87.755240,0.0,0.0,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,87629.997253,87629.997253,0.0,0.0


In [98]:
SMA_trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

In [99]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['SMAEntry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['SMA Portfolio Holdings']
        share_size = row['SMA Entry/Exit Position']
        entry_share_price = row['Close']

    elif row['SMAEntry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['Close'] * row['SMA Entry/Exit Position'])
        exit_share_price = row['Close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        SMA_trade_evaluation_df = SMA_trade_evaluation_df.append(
            {
                'Stock': ticker,
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
SMA_trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


In [100]:
entry_exit_plot.opts(title=f'{ticker}: EMA & ATR Trailing Stop Entry/Exit', xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)

In [101]:
portfolio_analysis = pn.Column("#Entry/Exit Comparison",
                                pn.Row(
                                   entry_exit_plot.opts(title=f'{ticker}: EMA & ATR Trailing Stop Entry/Exit', xaxis=None), 
                                   SMAentry_exit_plot.opts(title=f'{ticker}: SMA Entry/Exit', xaxis=None)),
                                "#Portfolio Value Comparison",
                               pn.Row(
                                   portfolio_entry_exit_plot.opts(title=f"{ticker}: EMA & ATR Trailing Stop Portfolio Value", xaxis=None),
                                   SMAportfolio_entry_exit_plot.opts(title=f'{ticker}: SMA Portfolio Value',xaxis=None)),
                               portfolio_evaluation_df

)

In [102]:
portfolio_analysis.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] HoloViews(Overlay)
        [1] HoloViews(Overlay)
    [2] Markdown(str)
    [3] Row
        [0] HoloViews(Overlay)
        [1] HoloViews(Overlay)
    [4] DataFrame(DataFrame)